In [1]:
library(topGO)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: graph

Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain 

In [3]:
geneID2GO <- readMappings(file = "../Reference_Files/Annotation_Files/Cre_gene2go")
geneNames <- names(geneID2GO)

In [4]:
RunTopGO <- function(interesting_genes){
    geneList <- factor(as.integer(geneNames %in% interesting_genes))
    names(geneList) <- geneNames
    
    storage <- data.frame(i=integer(), Annotation = I(list()))
    
    ontology = c("MF", "BP", "CC")
    for (term in 1:length(ontology)) {
        #set up the GOdata and run the algorithms 
        GOdata <- new("topGOdata", ontology = ontology[term], allGenes = geneList, annot = annFUN.gene2GO, 
                      gene2GO = geneID2GO)
        resultFis <- runTest(GOdata, algorithm = "classic", statistic = "fisher")
        
        #Generate the results table 
        gene_table <- GenTable(GOdata, classic = resultFis, orderBy = "classic", topNodes = length(usedGO(GOdata)), numChar=10000)
        gene_table$classic <- as.numeric(gene_table$classic)
        #just get the significant genes
        gene_table <- gene_table[gene_table$classic <= 0.01,]
        gene_table$adjusted_p <- p.adjust(gene_table$classic, method = "BH")
        
        #store gene table
        gene_table <- gene_table[order(gene_table$classic, -gene_table$Significant),] 
        storage[1, "i"] <- rownum
        if (nrow(gene_table) > 0){
            gene_table$component <- ontology[term]
            storage[[1, "Annotation"]] <- rbind(storage[[1, "Annotation"]], gene_table)
        }
    }
    return(storage)
    
}

In [5]:
summariseGO <- function(Annotation){
    summarise_counts <- ""
    summarise_pval <- ""
    
    AnnotationMF <- Annotation[Annotation$component == "MF", ]
    #AnnotationMF <- AnnotationMF[order(classic)] #ascending order
    AnnotationCC <- Annotation[Annotation$component == "CC", ]
    AnnotationBP <- Annotation[Annotation$component == "BP", ]

    summarise_counts <- paste(unlist(c("MF: ", AnnotationMF[order(-AnnotationMF$Significant),][1, "Term"], 
                                                      ", BP: ", AnnotationBP[order(-AnnotationBP$Significant),][1, "Term"],
                                                      ", CC: ", AnnotationCC[order(-AnnotationCC$Significant),][1, "Term"])), collapse = "")

    summarise_pval <- paste(unlist(c("MF: ", AnnotationMF[order(AnnotationMF$classic),][1, "Term"], 
                                                      ", BP: ", AnnotationBP[order(AnnotationBP$classic),][1, "Term"],
                                                      ", CC: ", AnnotationCC[order(AnnotationCC$classic),][1, "Term"])), collapse = "")
    return(data.frame(count_summary = summarise_counts,
                    pval_summary = summarise_pval))
    
}
